In [ ]:
!pip install -q langchain langgraph transformers bitsandbytes langchain-huggingface langchain-community chromadb

# baseline

將`關鍵字`比對換成`向量相似度`比對。

> 請將目前使用關鍵字比對的 route_by_query，改為使用向量相似度進行分類，並設一個合理的相似度門檻，根據檢索結果的分數判斷是否走 RAG 流程。  
例如用向量相似度及自訂 threshold 決定要不要分到 retriever。

> Hint：similarity_search_with_score(...)  
可參考去年的讀書會 R4：向量資料庫的基本操作

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n\n")]

# chromadb 預設使用的大型語言模型為 "all-MiniLM-L6-v2"，由於該大型語言模型不支持中文，所以將模型替換為 "infgrad/stella-base-zh-v3-1792d"，並對 embedding 進行量化
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)


persist_path = "documents"

# default distance : L2
collection_name_l2 = "naruto_collection_l2"
vectorstore_l2 = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name=collection_name_l2,
    collection_metadata={"hnsw:space": "l2"}
)

# cosine distance
collection_name_cosine = "naruto_collection_cosine"
vectorstore_cosine = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name=collection_name_cosine,
    collection_metadata={"hnsw:space": "cosine"}
)

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# 使用 4-bit 量化模型
model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 與 4-bit 模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

In [ ]:
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False # 僅返回生成的回應內容
)

Device set to use cuda:0


In [ ]:
from typing_extensions import TypedDict, List

# 定義 LangGraph 的 State 結構
class RAGState(TypedDict):
    query: str
    docs: List[Document]
    answer: str

### l2 distance

In [ ]:
# 定義 Retriever Node
def retrieve_node_l2(state: RAGState) -> RAGState:
    query = state["query"]
    # similarity_search 距離越小越相似
    docs = vectorstore_l2.similarity_search(query, k=3)
    return {"query": query, "docs": docs, "answer": ""}

# 定義 Generator Node（有檢索）
def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": docs, "answer": output}

# 定義 Direct Generator Node（不檢索）
def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": [], "answer": output}

# 定義 Route Node（決定走哪條路）
def route_by_similarity_score_l2(state: RAGState) -> str:
    query = state["query"]
    top = vectorstore_l2.similarity_search_with_score(query, k=3)
    _, distance = top[0]
    cosine_sim = 1 - (distance**2) / 2
    print(f"route:       l2 distance = {distance:.4f}")
    print(f"route: cosine similarity = {cosine_sim:.4f}")
    choice = "naruto" if cosine_sim > 0.5 else "general"
    print(f"跑到 → {choice}")
    return choice

In [ ]:
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 建立 LangGraph 流程圖
graph_builder_l2 = StateGraph(RAGState)

graph_builder_l2.set_entry_point("condition")
graph_builder_l2.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder_l2.add_node("retriever", RunnableLambda(retrieve_node_l2))
graph_builder_l2.add_node("generator", RunnableLambda(generate_node))
graph_builder_l2.add_node("direct_generator", RunnableLambda(direct_generate_node))

graph_builder_l2.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_similarity_score_l2),
    path_map={"naruto": "retriever", "general": "direct_generator"}
)

graph_builder_l2.add_edge("retriever", "generator")
graph_builder_l2.add_edge("generator", END)
graph_builder_l2.add_edge("direct_generator", END)

# 編譯 Graph
graph_l2 = graph_builder_l2.compile()

In [ ]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    # 設定初始 State
    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": ""
    }
    # 呼叫 LangGraph
    result = graph_l2.invoke(init_state)
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
route:       l2 distance = 0.5954
route: cosine similarity = 0.8227
跑到 → naruto


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
route:       l2 distance = 0.5204
route: cosine similarity = 0.8646
跑到 → naruto
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?
route:       l2 distance = 0.5047
route: cosine similarity = 0.8726
跑到 → naruto
回答： 第四代火影波風湊的徒弟有旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
route:       l2 distance = 1.2910
route: cosine similarity = 0.1666
跑到 → general
回答： 相對論是由愛因斯坦（Albert Einstein）在 1905 年提出的。他在 1905 年發表了一篇論文，其中包含了「特殊相對論」（Special Theory of Relativity），以及「廣義相對論」（General Theory of Relativity）的概念。

使用者: q
掰啦！


### cosine distance

In [ ]:
# 定義 Retriever Node
def retrieve_node_cosine(state: RAGState) -> RAGState:
    query = state["query"]
    # similarity_search 距離越小越相似
    docs = vectorstore_cosine.similarity_search(query, k=3)
    return {"query": query, "docs": docs, "answer": ""}

# 定義 Generator Node（有檢索）
def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": docs, "answer": output}

# 定義 Direct Generator Node（不檢索）
def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": [], "answer": output}

# 定義 Route Node（決定走哪條路）
def route_by_similarity_score_cosine(state: RAGState) -> str:
    query = state["query"]
    top = vectorstore_cosine.similarity_search_with_score(query, k=3)
    _, distance = top[0]
    cosine_sim = 1 - distance
    print(f"route:  cosine distance = {distance:.4f}")
    print(f"route: cosine similarty = {cosine_sim:.4f}")
    choice = "naruto" if cosine_sim > 0.5 else "general"
    print(f"跑到 → {choice}")
    return choice

In [ ]:
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 建立 LangGraph 流程圖
graph_builder_cosine = StateGraph(RAGState)

graph_builder_cosine.set_entry_point("condition")
graph_builder_cosine.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder_cosine.add_node("retriever", RunnableLambda(retrieve_node_cosine))
graph_builder_cosine.add_node("generator", RunnableLambda(generate_node))
graph_builder_cosine.add_node("direct_generator", RunnableLambda(direct_generate_node))

graph_builder_cosine.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_similarity_score_cosine),
    path_map={"naruto": "retriever", "general": "direct_generator"}
)

graph_builder_cosine.add_edge("retriever", "generator")
graph_builder_cosine.add_edge("generator", END)
graph_builder_cosine.add_edge("direct_generator", END)

# 編譯 Graph
graph_cosine = graph_builder_cosine.compile()

In [ ]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    # 設定初始 State
    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": ""
    }
    # 呼叫 LangGraph
    result = graph_cosine.invoke(init_state)
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
route:  cosine distance = 0.2977
route: cosine similarty = 0.7023
跑到 → naruto
回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
route:  cosine distance = 0.2602
route: cosine similarty = 0.7398
跑到 → naruto
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?
route:  cosine distance = 0.2524
route: cosine similarty = 0.7476
跑到 → naruto
回答： 第四代火影波風湊的徒弟有旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
route:  cosine distance = 0.6455
route: cosine similarty = 0.3545
跑到 → general
回答： 相對論是由愛因斯坦（Albert Einstein）在1905年提出的。

使用者: q
掰啦！


# advance

改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。

> 請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

> Hint：
```
class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str
```



In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n\n")]

# chromadb 預設使用的大型語言模型為 "all-MiniLM-L6-v2"，由於該大型語言模型不支持中文，所以將模型替換為 "infgrad/stella-base-zh-v3-1792d"，並對 embedding 進行量化
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model
)

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# 使用 4-bit 量化模型
model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 與 4-bit 模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

In [ ]:
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False # 僅返回生成的回應內容
)

Device set to use cuda:0


In [ ]:
class MultiTurnRAGState(TypedDict):
    history: List[str]
    query: str
    docs: List[Document]
    answer: str

In [ ]:
def retrieve_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = state["query"]
    history = state["history"]
    combined = "\n".join(history + [query])
    print("retrieve combined query:", repr(combined))
    docs = vectorstore.similarity_search(combined, k=3)
    return {"history": history, "query": query, "docs": docs, "answer": ""}

def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, history, docs = state["query"], state["history"], state["docs"]
    context = "\n".join(d.page_content for d in docs)
    history_text = "\n".join(f"輪 {i+1}: {q}" for i, q in enumerate(history))
    prompt = f"""你是一個知識型助手，請根據以下內容與對話歷史回答問題：

    內容：
    {context}

    對話歷史：
    {history_text}

    目前問題：
    {query}

    回答：
    """
    output = generator(prompt, max_new_tokens=300)[0]["generated_text"]
    return {
        "history": history + [query],
        "query": query,
        "docs": docs,
        "answer": output,
    }

def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, history = state["query"], state["history"]
    history_text = "\n".join(f"輪 {i+1}: {q}" for i, q in enumerate(history))
    prompt = f"""請參考以下對話歷史，回答當前問題：

    對話歷史：
    {history_text}

    目前問題：
    {query}

    回答：
    """
    output = generator(prompt, max_new_tokens=300)[0]["generated_text"]
    return {
        "history": history + [query],
        "query": query,
        "docs": [],
        "answer": output,
    }

def route_by_similarity_score(state: MultiTurnRAGState) -> str:
    query = state["query"]
    top = vectorstore.similarity_search_with_score(query, k=1)
    _, distance = top[0]
    cosine_sim = 1 - (distance**2) / 2
    print(f"route: cosine_sim = {cosine_sim:.4f}")
    choice = "naruto" if cosine_sim > 0.5 else "general"
    print(f"跑到 → {choice}")
    return choice

In [ ]:
graph_builder = StateGraph(MultiTurnRAGState)

graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

graph_builder.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_similarity_score),
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    },
)

graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)
graph = graph_builder.compile()

In [ ]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {"history": global_history, "query": user_input}
    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("AI 助理:", answer)
    print("===" * 60, "\n")

    global_history = result["history"]

開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
route: cosine_sim = 0.8092
跑到 → retriever
retrieve combined query: '第四代火影是誰?'
AI 助理: 第四代火影是波風湊。

使用者: 他的師父是誰?
route: cosine_sim = 0.5529
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?'
AI 助理: 第四代火影的師父是自來也。

使用者: 他的徒弟有哪些人?
route: cosine_sim = 0.6542
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?\n他的徒弟有哪些人?'
AI 助理: 他的徒弟有以下人：旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是他發明的嗎?
route: cosine_sim = 0.1118
跑到 → general
AI 助理: 相對論不是第四代火影所發明的。相對論是物理學家阿爾伯特·愛因斯坦在1905年提出的，他提出了廣義相對論，後人又提出狹義相對論。相對論主要是研究加速度和重力的關係，以及加速度和時間、空間的關係。

使用者: q
掰啦！
